## Importing the goodies 

In [1]:
import matplotlib.pyplot as plt
from IPython.display import clear_output
import numpy as np
import pyautogui
from PIL import Image 

from Xlib import display
import numpy as np
import matplotlib.pyplot as plt
import time
from mss.linux import MSS as mss
import mss.tools
import cv2
import pandas as pd 
import os

import torch
import torch.nn as nn 
import  torchvision.transforms as transforms

from torchvision.transforms import ToTensor
import torch.nn.functional as F

In [2]:

def countdown(sec):
    for i in range(sec):
        print(sec - i)
        time.sleep(1)

def find_mouse(x = False , y = False):
    
    data = display.Display().screen().root.query_pointer()._data
    res = [data["root_x"], data["root_y"]]
    if x == True and y == True:
        return res
    elif x == True and y == False:
        return res[0]
    else:
        return res[1]


def screen_record_efficient(top = 225, left = 0, width = 800, height = 260, resize_width = 80, resize_height = 25):
  
    mon = {"top": top, "left": left, "width": width, "height": height}
    sct = mss.mss()
    img = np.array(sct.grab(mon))
    img = cv2.resize(img, (resize_width, resize_height))
    img = cv2.cvtColor(img, cv2.COLOR_RGBA2RGB)

    sct.close()
    return img



def flex(np_img, name = "foo"):
    cv2.imshow(name, np_img)
    
def flex_with_steer(np_img, steer = 0, name = "foo"):
    cv2.imshow(name, np_img)
    
    
def canny(gray):
    im = cv2.Canny(gray,50,500,apertureSize = 3)
    return im

def fill_voids(img_np):
    kernel = np.ones((2,2),np.uint8)
    img = cv2.morphologyEx(img_np, cv2.MORPH_CLOSE, kernel, iterations = 1)
    return img
def move_mouse_co_ord_x(x_val , time = 0.001):
    
    current_y = find_mouse(y = True)
    pyautogui.moveTo(x_val, current_y, duration= time )


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.upper_conv1 = nn.Conv2d(1, 16, 3, padding = 1)
        self.upper_conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        self.upper_conv3 = nn.Conv2d(32, 32, 3, padding = 1)

        self.pool = nn.MaxPool2d(3, 3)
        
        self.upper_fc1 = nn.Linear(256,128)
        self.upper_fc2 = nn.Linear(128, 64)

################################################################# had to do this 

        self.lower_conv1 = nn.Conv2d(3, 16, 3, padding = 1)
        self.lower_conv2 = nn.Conv2d(16, 32, 3, padding = 1)
        self.lower_conv3 = nn.Conv2d(32, 64, 3, padding = 1)
        
        self.lower_fc1 = nn.Linear(256,128)
        self.lower_fc2 = nn.Linear(128, 64)

        ## meet and greet at at 64, 64, concat to 128 

        self.combined_fc1 = nn.Linear(128, 128)
        self.combined_fc2 = nn.Linear(128, 64)
        self.combined_fc3 = nn.Linear(64, 1)


    def forward(self, x, cuda = False):  ## set cuda to False because no local GPU

        (upper, lower)  = x   ## unpack 

        if cuda == True:
            upper = upper.cuda()
            lower = lower.cuda()

        x_upper = self.pool(F.relu(self.upper_conv1(upper)))
        x_upper = self.pool(F.relu(self.upper_conv2(x_upper)))
        x_upper = self.pool(F.relu(self.upper_conv3(x_upper)))

        x_upper = x_upper.view(x_upper.size(0), -1)
        x_upper = F.relu(self.upper_fc1(x_upper))
        x_upper = F.relu(self.upper_fc2(x_upper))

################################# one more border, please dont mess this part up 

        x_lower = self.pool(F.relu(self.lower_conv1(lower)))
        x_lower = self.pool(F.relu(self.lower_conv2(x_lower)))
        x_lower = self.pool(F.relu(self.lower_conv3(x_lower)))

        x_lower = x_lower.view(x_lower.size(0), -1)
        x_lower = F.relu(self.lower_fc1(x_lower))
        x_lower = F.relu(self.lower_fc2(x_lower))
    
        x = torch.cat((x_upper ,x_lower), dim=1)

        x = F.relu(self.combined_fc1(x))
        x = F.relu(self.combined_fc2(x))
        x = self.combined_fc3(x)


        return x

drunk_driver = Net()
drunk_driver.load_state_dict(torch.load("torch/drunk_driver_ensemble_2.pt", map_location=torch.device('cpu')))

<All keys matched successfully>

In [4]:
transform =  transforms.Compose([          
                                transforms.ToPILImage(),
                                ToTensor()
                                ])

In [24]:


def running_in_the_90s(flex_mode = False, segment_road = True):
    
    print ("running...")
    print ("drag mouse to the corner of the screen to terminate -")
    
    while True:
        
        
        upper_raw = screen_record_efficient(top = 225 + 200 + 45,   ## main + lower + remove sky
                                      width = 800, 
                                      height = 142, 
                                      resize_width = 128, 
                                      resize_height = 64)  ## FPV TRACK VIEW  
    
        
        lower_raw = screen_record_efficient(top = 225 ,   ## main + lower + remove sky
                                              width = 800, 
                                              height = 142, 
                                              resize_width = 64, 
                                              resize_height = 64)  ## FPV TRACK VIEW  
        
         
        mouse =find_mouse(x = True)
        steer = 400 - mouse
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break

        if segment_road == True:
            upper_seg = canny(upper_raw)

        if flex_mode == True:
            flex(upper_seg , name = "upper")
            flex(lower_raw, name = "lower")
            
        (upper , lower)  = (transform(upper_seg).unsqueeze(0), transform(lower_raw).unsqueeze(0))
        
        pred = int(drunk_driver((upper,lower)).detach().item())
        steer = 400 - pred 
        
        try:
            move_mouse_co_ord_x(x_val = steer , time = 0.001)
        except:
#             cv2.destroyAllWindows()
            print("stopped")
            break
            


In [25]:
countdown(2)
running_in_the_90s(flex_mode = True, segment_road = True)  

2
1
running...
drag mouse to the corner of the screen to terminate -
stopped


In [26]:
cv2.destroyAllWindows()
